# Frequent Subtree Counting in Random Forests

The goal of this project is to compress the generated source code of decision tree classifiers on embedded devices.
Therefore, as a first step, we investigate for several trained random forests, whether they have certain frequent subtrees in common.
Such subtrees might be implemented by a function which is called several times in the corresponding places of the decision trees. 
This can decrease the code size of the generated embedded-c source files and executables.

## Datasets
There are several datasets.
At the moment, however, I'll experiment only with 'adult' and 'wine-quality'.
The data are given as json files of the following names:

In [1]:
ls forests/*/text/*.json

forests/adult/text/DT_10.json
forests/adult/text/DT_15.json
forests/adult/text/DT_1.json
forests/adult/text/DT_20.json
forests/adult/text/DT_5.json
forests/adult/text/ET_10.json
forests/adult/text/ET_15.json
forests/adult/text/ET_1.json
forests/adult/text/ET_20.json
forests/adult/text/ET_5.json
forests/adult/text/RF_10.json
forests/adult/text/RF_15.json
forests/adult/text/RF_1.json
forests/adult/text/RF_20.json
forests/adult/text/RF_5.json
forests/satlog/text/RF_10.json
forests/satlog/text/RF_10_pruned_with_sigma_0_0.json
forests/satlog/text/RF_10_pruned_with_sigma_0_1.json
forests/satlog/text/RF_10_pruned_with_sigma_0_2.json
forests/satlog/text/RF_10_pruned_with_sigma_0_3.json
forests/satlog/text/RF_15.json
forests/satlog/text/RF_15_pruned_with_sigma_0_0.json
forests/satlog/text/RF_15_pruned_with_sigma_0_1.json
forests/satlog/text/RF_15_pruned_with_sigma_0_2.json
forests/satlog/text/RF_15_pruned_with_sigma_0_3.json
forests/satlog/text/RF_20.json
forests/satlog/text/RF_20_pruned_with_s

The filenames XX_n.json mean:
- XX:
  - DT decision tree
  - RF random forest with 25 trees
  - ET 'extremely random trees'
- n: depth of the trees

Each tree given here (whether as a decision tree or as a component in a random forest) is a rooted, binary, ordered tree.

## Convert the Data

We have written a few scripts to convert from json to the format required by the frequent subgraph mining software.

In [2]:
ls ./*.py

./json2graphNoLeafEdges.py
./json2graphNoLeafEdgesWithSplitValues.py
./json2graphWithLeafEdges.py
./json2graphWithLeafEdgesWithSplitValues.py


See the documentation of the scripts to check what they are doing:

In [2]:
%%bash
for f in ./*.py; do
    echo ${f}
    grep '^#' < ${f}
    echo
done

./file_splitting_into_train_and_test.py
#!/usr/bin/env python

./json2graphNoLeafEdges.py
## This script creates a graph database from the decision trees in Sebastians json Format as follows:
# - the root vertex of the tree has index 1 (counting starts with 1)
# - each vertex is labeled by its split feature or by 'leaf'
# - each edge is labeled either 'leftChild' or 'rightChild'
# - there are no edges containing 'leaf' vertices
#
# It follows that the connected components resulting from a single decision tree are several isolated vertices labeled 'leaf' 
# and a tree containing all the split vertices.

./json2graphNoLeafEdgesWithSplitValues.py
## This script creates a graph database from the decision trees in Sebastians json Format as follows:
# - the root vertex of the tree has index 1 (counting starts with 1)
# - each vertex is labeled by its split feature or by 'leaf'
# - each edge is labeled either 'leftChild' or 'rightChild'
# - there are no edges containing 'leaf' vertices
#
# It

In [3]:
%%bash
mkdir forests/spambase/WithLeafEdges/
mkdir forests/spambase/NoLeafEdges/

mkdir forests/satlog/WithLeafEdges/
mkdir forests/satlog/NoLeafEdges/

mkdir: cannot create directory ‘forests/spambase/WithLeafEdges/’: File exists
mkdir: cannot create directory ‘forests/spambase/NoLeafEdges/’: File exists
mkdir: cannot create directory ‘forests/satlog/WithLeafEdges/’: File exists
mkdir: cannot create directory ‘forests/satlog/NoLeafEdges/’: File exists


CalledProcessError: Command 'b'mkdir forests/spambase/WithLeafEdges/\nmkdir forests/spambase/NoLeafEdges/\n\nmkdir forests/satlog/WithLeafEdges/\nmkdir forests/satlog/NoLeafEdges/\n'' returned non-zero exit status 1.

In [4]:
%%bash
for dataset in spambase satlog; do
    for f in forests/${dataset}/text/*.json; do
        echo ${f} '->' `basename ${f} .json`.graph
        python json2graphWithLeafEdges.py ${f} > forests/${dataset}/WithLeafEdges/`basename ${f} .json`.graph
        python json2graphNoLeafEdges.py ${f} > forests/${dataset}/NoLeafEdges/`basename ${f} .json`.graph
    done
done

forests/spambase/text/RF_10.json -> RF_10.graph
forests/spambase/text/RF_10_pruned_with_sigma_0_0.json -> RF_10_pruned_with_sigma_0_0.graph
forests/spambase/text/RF_10_pruned_with_sigma_0_1.json -> RF_10_pruned_with_sigma_0_1.graph
forests/spambase/text/RF_10_pruned_with_sigma_0_2.json -> RF_10_pruned_with_sigma_0_2.graph
forests/spambase/text/RF_10_pruned_with_sigma_0_3.json -> RF_10_pruned_with_sigma_0_3.graph
forests/spambase/text/RF_15.json -> RF_15.graph
forests/spambase/text/RF_15_pruned_with_sigma_0_0.json -> RF_15_pruned_with_sigma_0_0.graph
forests/spambase/text/RF_15_pruned_with_sigma_0_1.json -> RF_15_pruned_with_sigma_0_1.graph
forests/spambase/text/RF_15_pruned_with_sigma_0_2.json -> RF_15_pruned_with_sigma_0_2.graph
forests/spambase/text/RF_15_pruned_with_sigma_0_3.json -> RF_15_pruned_with_sigma_0_3.graph
forests/spambase/text/RF_20.json -> RF_20.graph
forests/spambase/text/RF_20_pruned_with_sigma_0_0.json -> RF_20_pruned_with_sigma_0_0.graph
forests/spambase/text/RF_20_

### Next Steps
The resulting files are used in the notebooks whose starting numbers are larger than 1 